## This notebook contains several feature extraction techniques which are used to extract features from the facial images and then passed to a CNN model to compare their results

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras import regularizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
np.random.seed(42)

# loading data

In [ ]:
os.listdir('/kaggle/input/ckplus')

In [ ]:
DATADIR = r'/kaggle/input/ckplus/CK+48'

In [ ]:
CATEGORIES = os.listdir(DATADIR)
CATEGORIES

In [ ]:
def load_data():
    DATADIR = r'/kaggle/input/ckplus/CK+48'
    data = []
    # loading training data
    for category in CATEGORIES:
        # create path to image of respective expression
        path = os.path.join(DATADIR, category)
        # get the classification  for each expression 
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), 0)
            data.append([img_array, class_num])
            
    return data

In [ ]:
data = load_data()

In [ ]:
len(data)

# Lets Visualize some images 

In [ ]:
L = 4
W = 4
fig, axes = plt.subplots(L, W, figsize = (15,15))
axes = axes.ravel()

for i in range(0, L * W):  
    sample = random.choice(data)
    axes[i].set_title("Expression = "+str(CATEGORIES[sample[1]]))
    axes[i].imshow(sample[0], cmap='gray')
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)

# Creating training and testing data

In [ ]:
X = np.array([ x[0] for x in data])
y = np.array([Y[1] for Y in data])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("-------------------------------")
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
# reshaping y_train and y_test
y_train = np.reshape(y_train, (len(y_train),1))
y_test  = np.reshape(y_test , (len(y_test ),1))

print("After reshaping")
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
X_train_Gabor  = X_train
X_test_Gabor = X_test

## Adding color channel 

In [ ]:
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

print("After adding color channel")
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

## Normalizing pixel values 

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## Converting single values to category array 

In [ ]:
y_train[0]

In [ ]:
y_train_SVM = y_train
y_test_SVM = y_test

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
y_train[0]

In [ ]:
y_train.shape, y_test.shape

# Feature Extraction

# [HOG technqiue](https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/) 
For more info on implementation [visit here](https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/)

In [ ]:
from skimage.transform import resize
from skimage.feature import hog

### How Hog looks ?

In [ ]:
plt.figure(figsize=(10, 10))


plt.subplot(1,2,1)
img = random.choice(X_train)
# first image needs to be resized before passing it to HOG descriptor
resized_img =  resize(img, (128, 64))
plt.title("Original image")
plt.imshow(img, cmap='gray')
fd, hog_image = hog(
    resized_img, 
    orientations=9, 
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2), 
    visualize=True, 
    multichannel=True
)
plt.subplot(1,2,2)
plt.title("HOG")
plt.imshow(resize(hog_image, (48, 48)), cmap='gray')
plt.axis('off')

### Creating Feature Vectors for training and testing 

In [ ]:
def Create_Hog_features(data):
    Feature_data = np.zeros((len(data),48,48))

    for i in range(len(data)):
        img = data[i]
        resized_img = resize(img, (128, 64))
        fd, hog_image = hog(
            resized_img, 
            orientations=9, 
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), 
            visualize=True, 
            multichannel=True
        )
        Feature_data[i] = resize(hog_image, (48, 48))
    return Feature_data

In [ ]:
Feature_X_train = Create_Hog_features(X_train)
Feature_X_train.shape

In [ ]:
plt.imshow(random.choice(Feature_X_train), cmap='gray')
plt.axis('off')

In [ ]:
# doing same for test data 
Feature_X_test = Create_Hog_features(X_test)

Feature_X_test.shape

In [ ]:
plt.imshow(random.choice(Feature_X_test), cmap='gray')
plt.axis('off')

In [ ]:
# Again adding color channel as it got removed while converting img to hog img
X_train_HOG = np.expand_dims(Feature_X_train, axis=3)
X_test_HOG = np.expand_dims(Feature_X_test, axis=3)

print("After adding color channel")
print("X_train_HOG shape: ", X_train_HOG.shape)
print("X_test_HOG shape: ", X_test_HOG.shape)

In [ ]:
print("X_train_HOG shape: ", X_train_HOG.shape)
print("y_train shape: ", y_train.shape)
print("X_test_HOG shape: ", X_test_HOG.shape)
print("y_test shape: ", y_test.shape)

### Data Augmentation

In [ ]:
# train_datagen = ImageDataGenerator(
#     rotation_range=25, width_shift_range=0.1,
#     height_shift_range=0.1, shear_range=0.2, 
#     zoom_range=0.2,horizontal_flip=True, 
#     fill_mode="nearest"
# )

### Training and testing the HOG - CNN model

Referred from [here](https://www.kaggle.com/milan400/human-emotion-detection-by-using-cnn#Creating-Model)

In [ ]:
def create_model(input_shape=None):
    if input_shape is None :
        input_shape=(48,48,1)

    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=input_shape, padding='same', activation = 'relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (5, 5), padding='same', activation = 'relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation = 'softmax'))
    
    return model 

In [ ]:
es = EarlyStopping(
    monitor='val_accuracy', min_delta=0.0001, patience=10, verbose=2,
    mode='max', baseline=None, restore_best_weights=True
)
lr = ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=5, verbose=2,
    mode='max', min_delta=1e-5, cooldown=0, min_lr=0
)

callbacks = [es, lr]

In [ ]:
HOG_model = create_model()

In [ ]:
HOG_model.summary()

In [ ]:
HOG_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam' )

In [ ]:
HOG_history = HOG_model.fit(X_train_HOG, y_train, batch_size=8 , epochs=50, validation_data = (X_test_HOG, y_test))

In [ ]:
def plot_performance(history):
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 1, 1)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')

    plt.legend()
    plt.grid()
    plt.title('train and val loss evolution')

    plt.subplot(2, 1, 2)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='val')

    plt.legend()
    plt.grid()
    plt.title('train and val accuracy')

In [ ]:
plot_performance(HOG_history)

In [ ]:
acc = []

In [ ]:
HOG_acc = HOG_model.evaluate(X_test_HOG, y_test, verbose = 0)[1]
acc.append(HOG_acc)
print("HOG Accuracy :",HOG_model.evaluate(X_test_HOG, y_test, verbose = 0)[1])

In [ ]:
HOG_model.save('HOG_model.h5')

# LBP technqiue
Implementation Referred from [here](https://github.com/salonibhatiadutta/To-get-the-local-binary-pattern-LBP-of-image-and-draw-its-histogram/blob/master/gray_image_conversion.ipynb)

In [ ]:
def Binarypattern(im):                               # creating function to get local binary pattern
    img= np.zeros_like(im)
    n=3                                              # taking kernel of size 3*3
    for i in range(0,im.shape[0]-n):                 # for image height
        for j in range(0,im.shape[1]-n):               # for image width
            x  = im[i:i+n,j:j+n]                     # reading the entire image in 3*3 format
            center       = x[1,1]                    # taking the center value for 3*3 kernel
            img1        = (x >= center)*1.0          # checking if neighbouring values of center value is greater or less than center value
            img1_vector = img1.T.flatten()           # getting the image pixel values 
            img1_vector = np.delete(img1_vector,4)  
            digit = np.where(img1_vector)[0]         
            if len(digit) >= 1:                     # converting the neighbouring pixels according to center pixel value
                num = np.sum(2**digit)              # if n> center assign 1 and if n<center assign 0
            else:                                    # if 1 then multiply by 2^digit and if 0 then making value 0 and aggregating all the values of kernel to get new center value
                num = 0
            img[i+1,j+1] = num
    return(img)

### How LBP looks ?

In [ ]:
plt.figure(figsize = (10,10))

plt.subplot(1,2,1)
img = random.choice(X_train)
plt.title("Original  image")
plt.imshow(img, cmap='gray')

plt.subplot(1,2,2)
plt.title("LBP")
imgLBP=Binarypattern(img)             # calling the LBP function using gray image
plt.imshow(imgLBP, cmap='gray')
plt.axis('off')

### Creating Feature Vectors for training and testing 

In [ ]:
X_train.shape

In [ ]:
def create_LBP_features(data):
    Feature_data = np.zeros(data.shape)

    for i in range(len(data)):
        img = data[i]
        imgLBP=Binarypattern(img)  
        Feature_data[i] = imgLBP
    
    return Feature_data

In [ ]:
Feature_X_train = create_LBP_features(X_train)

In [ ]:
Feature_X_train.shape

In [ ]:
img = random.choice(Feature_X_train)
plt.imshow(img, cmap='gray')

In [ ]:
Feature_X_test = create_LBP_features(X_test)
Feature_X_test.shape

In [ ]:
img = random.choice(Feature_X_test)
plt.imshow(img, cmap='gray')

### Training and testing LBP-CNN model

In [ ]:
LBP_model = create_model()
LBP_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam' )

In [ ]:
LBP_history = LBP_model.fit(Feature_X_train, y_train, batch_size=8 , epochs=50, validation_data = (Feature_X_test, y_test) )

In [ ]:
plot_performance(LBP_history)

In [ ]:
LBP_acc = LBP_model.evaluate(Feature_X_test, y_test, verbose = 0)[1]
acc.append(LBP_acc)
print("LBP Accuracy :",LBP_model.evaluate(Feature_X_test, y_test, verbose = 0)[1])

In [ ]:
LBP_model.save('LBP_model.h5')

# SIFT 

### Let's see some SIFT features 

In [ ]:
L = 3
W = 3
fig, axes = plt.subplots(L, W, figsize = (15,15))
axes = axes.ravel()

for i in range(0, L * W):  
    sample = random.choice(data)
    image8bit = cv2.normalize(sample[0], None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(image8bit,None)

    img = cv2.drawKeypoints(image=image8bit, outImage=sample[0], keypoints = kp, flags = 4, color = (255, 0, 0))
    axes[i].set_title("Expression = "+str(CATEGORIES[sample[1]]))
    axes[i].imshow(img, cmap='gray')
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)

In [ ]:
img.shape

so to create a feature vector we will have to create an empty array of shape (len(data), 48,48,3) 

### Creating Feature Vectors for training and testing 

In [ ]:
def create_SIFT_features(data):
    Feature_data = np.zeros((len(data),48,48,3))

    for i in range(len(data)):
        img = data[i]
        image8bit = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        sift = cv2.SIFT_create()
        kp, des = sift.detectAndCompute(image8bit,None)

        img = cv2.drawKeypoints(image=image8bit, outImage=img, keypoints = kp, flags = 4, color = (255, 0, 0))
        Feature_data[i] = img/255.0

        
    return Feature_data 

In [ ]:
X_train_SIFT = create_SIFT_features(X_train) 
X_train_SIFT.shape

In [ ]:
plt.imshow(X_train_SIFT[0], cmap='gray')

In [ ]:
X_test_SIFT = create_SIFT_features(X_test) 
X_test_SIFT.shape

In [ ]:
plt.imshow(X_test_SIFT[0], cmap='gray')

In [ ]:
SIFT_model = create_model(input_shape=(48,48,3))
SIFT_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam' )

In [ ]:
SIFT_history = SIFT_model.fit(X_train_SIFT, y_train, batch_size=8 , epochs=50, validation_data = (X_test_SIFT, y_test),  )

In [ ]:
plot_performance(SIFT_history)

In [ ]:
SIFT_acc = SIFT_model.evaluate(X_test_SIFT, y_test, verbose = 0)[1]
acc.append(SIFT_acc)
print("SIFT Accuracy :",SIFT_model.evaluate(X_test_SIFT, y_test, verbose = 0)[1])

In [ ]:
SIFT_model.save('SIFT_model.h5')

# Gabor Filters 
with help of my friend : https://www.kaggle.com/sashankmvv/gabor-cnn-ck/notebook

In [ ]:
def Gabor_filter(K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get half size
    d = K_size // 2

    # prepare kernel
    gabor = np.zeros((K_size, K_size), dtype=np.float32)

    # each value
    for y in range(K_size):
        for x in range(K_size):
            # distance from center
            px = x - d
            py = y - d

            # degree -> radian
            theta = angle / 180. * np.pi

            # get kernel x
            _x = np.cos(theta) * px + np.sin(theta) * py

            # get kernel y
            _y = -np.sin(theta) * px + np.cos(theta) * py

            # fill kernel
            gabor[y, x] = np.exp(-(_x**2 + Gamma**2 * _y**2) / (2 * Sigma**2)) * np.cos(2*np.pi*_x/Lambda + Psi)

    # kernel normalization
    gabor /= np.sum(np.abs(gabor))

    return gabor


# Use Gabor filter to act on the image
def Gabor_filtering(gray, K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get shape
    H, W = gray.shape

    # padding
    gray = np.pad(gray, (K_size//2, K_size//2), 'edge')

    # prepare out image
    out = np.zeros((H, W), dtype=np.float32)

    # get gabor filter
    gabor = Gabor_filter(K_size=K_size, Sigma=Sigma, Gamma=Gamma, Lambda=Lambda, Psi=0, angle=angle)

    # filtering
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(gray[y : y + K_size, x : x + K_size] * gabor)

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out


# Use 6 Gabor filters with different angles to perform feature extraction on the image
def Gabor_process(img):
#     print(img.shape)
    # get shape
    H, W = img.shape

    # gray scale
#     gray = BGR2GRAY(img).astype(np.float32)

    # define angle
    #As = [0, 45, 90, 135]
    As = [0,30,60,90,120,150]

    # prepare pyplot
#     plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0.2)

    out = np.zeros([H, W], dtype=np.float32)

    # each angle
    for i, A in enumerate(As):
    
        # gabor filtering
        _out = Gabor_filtering(img, K_size=11, Sigma=1.5, Gamma=1.2, Lambda=3, angle=A)
         

        # add gabor filtered image
        out += _out
        

    # scale normalization
    out = out /out.max()*255
    out = out.astype(np.uint8)

    return out

In [ ]:
def create_Gabor_features(data):
    Feature_data = np.zeros((len(data),48,48,1))

    for i in range(len(data)):
        img = data[i]
        out = Gabor_process(img)
        out = np.expand_dims(out , axis = 2) # adding color channel
        Feature_data[i] = out/255.00

        
    return Feature_data 

In [ ]:
X_train.shape

In [ ]:
plt.imshow(X_train_Gabor[0]/255.0, cmap ='gray')

In [ ]:
X_train_Gabor=create_Gabor_features(X_train_Gabor)
X_test_Gabor=create_Gabor_features(X_test_Gabor)

X_train_Gabor.shape , X_test_Gabor.shape

In [ ]:
sample = random.randint(100,500)
plt.subplot(1,2,1)
plt.imshow(X_train[sample],cmap='gray')
plt.axis("off")
plt.subplot(1,2,2)
plt.imshow(X_train_Gabor[sample],cmap='gray')
plt.axis("off")

In [ ]:
X_train_Gabor.shape

In [ ]:
Gabor_model = create_model()
Gabor_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam' )

In [ ]:
Gabor_history = Gabor_model.fit(X_train_Gabor, y_train, batch_size=8 , epochs=50, validation_data = (X_test_Gabor, y_test) )

In [ ]:
plot_performance(Gabor_history)

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

In [ ]:
Gabor_acc = Gabor_model.evaluate(X_test_Gabor, y_test, verbose = 0)[1]
acc.append(Gabor_acc)
print("Gabor Accuracy :",Gabor_model.evaluate(X_test_Gabor, y_test, verbose = 0)[1])

In [ ]:
Gabor_model.save('Gabor_model.h5')

# Without Feature Extraction 

In [ ]:
WFE_model = create_model()
WFE_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam' )

In [ ]:
WFE_history = WFE_model.fit(X_train, y_train, batch_size=8 , epochs=50, validation_data = (X_test, y_test) ,callbacks = [callbacks])

In [ ]:
plot_performance(WFE_history)

In [ ]:
WFE_acc = WFE_model.evaluate(X_test, y_test, verbose = 0)[1]
acc.append(WFE_acc)
print("Without Feature extraction Accuracy :", WFE_acc)

In [ ]:
WFE_model.save('WFE_model.h5')

# Comparing accuracies 

In [ ]:
acc

In [ ]:
results = pd.DataFrame(acc, index=['HOG', 'LBP', 'SIFT', 'Gabor', 'Without Feature Extraction'], columns = ['Accuracies'])

In [ ]:
dfStyler = results.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

In [ ]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

In [ ]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
plt.show()

In [ ]:
y_pred=HOG_model.predict(X_test)
#print(y_pred)
y_test.shape

In [ ]:
from skimage import io
img = image.load_img('/kaggle/input/testimg/download.jpeg', grayscale=True, target_size=(48, 48))
show_img=image.load_img('/kaggle/input/testimg/download.jpeg', grayscale=False, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = HOG_model.predict(x)
#print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])